In [1]:
import pandas as pd
import numpy as np
import time # 코드 수행 시간

In [92]:
file_path='./datas/'
file_name=file_path+"h1b_train.csv"
train=pd.read_csv(file_name)
train=train.loc[(train['CASE_STATUS']=='CERTIFIED') | (train['CASE_STATUS']=='DENIED')]

In [93]:
def init_train_groupby () :
    train_groupby=pd.DataFrame({'COUNT': train.groupby(['EMPLOYER_NAME','SOC_NAME','JOB_TITLE','FULL_TIME_POSITION']).size().sort_values(ascending=False)}).reset_index()
    train_groupby["FREQUENCY"]=0
    return train_groupby


In [94]:
train_groupby=init_train_groupby()
train_groupby

,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,COUNT,FREQUENCY
0,INFOSYS LIMITED,Computer Systems Analysts,TECHNOLOGY LEAD - US,Y,8551,0
1,INFOSYS LIMITED,Computer Systems Analysts,TECHNOLOGY ANALYST - US,Y,8397,0
2,TATA CONSULTANCY SERVICES LIMITED,Computer Programmers,COMPUTER PROGRAMMER,Y,8041,0
3,INFOSYS LIMITED,COMPUTER SYSTEMS ANALYSTS,TECHNOLOGY LEAD - US,Y,5211,0
4,INFOSYS LIMITED,COMPUTER SYSTEMS ANALYSTS,TECHNOLOGY ANALYST - US,Y,4194,0
5,INFOSYS LIMITED,Computer Programmers,SYSTEMS ENGINEER - US,Y,3097,0
6,INFOSYS LIMITED,COMPUTER SYSTEMS ANALYST,TECHNOLOGY ANALYST - US,N,2798,0
7,TATA CONSULTANCY SERVICES LIMITED,COMPUTER PROGRAMMERS,DEVELOPER,Y,2758,0
8,TATA CONSULTANCY SERVICES LIMITED,COMPUTER PROGRAMMERS,DEVELOPER,N,2435,0
9,WIPRO LIMITED,Computer Programmers,PROGRAMMER ANALYST,Y,2345,0


In [95]:
train[train.SOC_NAME.isna()]

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
44,1333364,CERTIFIED,LARSEN & TOUBRO INFOTECH LIMITED,NaN,TECHNICAL CONSULTANT,Y,70762.0,2014.0,"HARLEYSVILLE, PENNSYLVANIA",-75.387121,40.279548
57,2902632,CERTIFIED,WIPRO LIMITED,NaN,DATABASE ADMINISTRATOR,Y,59654.0,2011.0,"IRVING, TEXAS",-96.948894,32.814018
80,1778133,DENIED,EMILY BOLL,NaN,FITNESS TRAINER,N,37440.0,2014.0,"HONOLULU, HAWAII",-157.858333,21.306944
137,2553233,CERTIFIED,BROADCOM CORPORATION,NaN,"ENGINEER, PRINCIPAL - SOFTWARE DEVELOPMENT",Y,142875.0,2012.0,"SUNNYVALE, CALIFORNIA",-122.036350,37.368830
426,2338705,CERTIFIED,"COMPUTERIZED ASSESSMENTS AND LEARNING, LLC",NaN,SENIOR JAVA WEB DEVELOPER/ PROGRAMMER,Y,50939.2,2012.0,"LAWRENCE, KANSAS",-95.235250,38.971669
447,2852120,CERTIFIED,WIPRO LIMITED,NaN,BUSINESS SYSTEM ANALYST,Y,72613.0,2011.0,"DUBLIN, OHIO",-83.114077,40.099229
1164,1687223,CERTIFIED,LARSEN & TOUBRO INFOTECH LIMITED,NaN,COMPUTER SYSTEMS ENGINEER,Y,45552.0,2014.0,"WARREN, NEW JERSEY",-74.500480,40.634249
1260,1426083,CERTIFIED,LARSEN & TOUBRO INFOTECH LIMITED,NaN,IT PROJECT MANAGER,Y,71053.0,2014.0,"FARMINGTON, CONNECTICUT",-72.795027,41.736030
1294,2877209,CERTIFIED,LARSEN & TOUBRO INFOTECH LIMITED,NaN,COMPUTER PROGRAMMER ANALYST,Y,75379.0,2011.0,"NEW BRUNSWICK, NEW JERSEY",-74.451819,40.486216
1535,1787362,CERTIFIED,UNIVERSITY OF MICHIGAN,NaN,FINANCIAL SPECIALIST ASSOCIATE,Y,28434.0,2013.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826


In [97]:
def match_soc_name(employer_name,job_title,full_time_position):
    """
    최빈값으로  socname 입력해주기
    EMPLOYER SAME -> 4 add
    JOB_TITLE -> 2 add
    FULL_TIME SAME -> 1 add
    가중치설정
    """
    train_groupby["FREQUENCY"]=0
    #print (employer_name,job_title,full_time_position)
    #print (type((employer_name)))
    train_groupby.loc[(train_groupby['EMPLOYER_NAME']==employer_name),'FREQUENCY'] +=4
    train_groupby.loc[(train_groupby['JOB_TITLE']==job_title),'FREQUENCY']+=2
    train_groupby.loc[(train_groupby['FULL_TIME_POSITION']==full_time_position),'FREQUENCY'] +=1
    return train_groupby.sort_values(['FREQUENCY','COUNT'],ascending=[False,False]).head(1).SOC_NAME.values[0]

In [98]:
start_time = time.time() 
train.loc[train['SOC_NAME'].isnull(),'SOC_NAME'] = train.apply(lambda x : match_soc_name(x['EMPLOYER_NAME'],x['JOB_TITLE'],x['FULL_TIME_POSITION']) if pd.isnull(x['SOC_NAME']) else "",axis=1)
end_time = time.time()

KeyboardInterrupt: 

In [ ]:
print (end_time-start_time)

In [ ]:
save_file=file_path+'h1b_train_fill_soc_na.csv'
train.to_csv(save_file,index=False)

In [85]:
train.head(50)

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1349742,CERTIFIED,"INTERNATIONAL SYSTEMS TECHNOLOGIES, INC.",Computer Programmers,PROGRAMMER ANALYST,Y,61589.0,2014.0,"SUNNYVALE, CALIFORNIA",-122.036350,37.368830
1,789205,CERTIFIED,GOOGLE INC.,NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS,SITE RELIABILITY ENGINEER,Y,79144.0,2015.0,"SAN FRANCISCO, CALIFORNIA",-122.419415,37.774929
2,1617140,CERTIFIED,WIPRO LIMITED,"Computer Occupations, All Other",PROJECT MANAGER,Y,73882.0,2014.0,"EL SEGUNDO, CALIFORNIA",-118.416465,33.919180
3,2757905,CERTIFIED,"CPO SERVICES, INC.",Orthotists and Prosthetists,STAFF PROSTHETIST,Y,61589.0,2011.0,"PEORIA, ILLINOIS",-89.588986,40.693649
5,693102,DENIED,"UNIVERSITY OF CALIFORNIA, SAN DIEGO","MEDICAL SCIENTISTS, EXCEPT EPIDEMIOLOGISTS",ASSISTANT PROJECT SCIENTIST,Y,42120.0,2015.0,"LA JOLLA, CALIFORNIA",-117.271272,32.832811
6,2662510,CERTIFIED,GOOGLE INC.,"Computer Software Engineers, Applications",SOFTWARE ENGINEER,Y,82035.0,2011.0,"MOUNTAIN VIEW, CALIFORNIA",-122.083851,37.386052
7,636434,CERTIFIED,"COMPREHAB, INC.",PHYSICAL THERAPISTS,PHYSICAL THERAPIST,N,64168.0,2016.0,"GASTONIA, NORTH CAROLINA",-81.187301,35.262082
8,2265757,CERTIFIED,JPMORGAN CHASE AND CO.,"Software Developers, Applications",APPLICATIONS DEVELOPER,Y,92726.0,2012.0,"COLUMBUS, OHIO",-82.998794,39.961176
9,2927315,CERTIFIED,"IRONBRIDGE CONSULTING, LLC","Information Security Analysts, Web Developers,...",SOFTWARE ENGINEER/ NETWORK ANALYST,Y,57866.0,2011.0,"ARLINGTON, VIRGINIA",-77.106770,38.879970
10,1696111,CERTIFIED,"AXON SOLUTIONS, INC.",Computer Systems Analysts,MANAGING CONSULTANT,Y,91998.0,2014.0,"UNIVERSAL CITY, CALIFORNIA",-118.350578,34.139769


In [74]:
type(train[train['SOC_NAME'].isna()].SOC_NAME)

pandas.core.series.Series